# Machine Learning

**Model**: a specification of a mathematical (or probabilistic) relationship between different variables. Depending on the selected model and problem, this model may poorly reflect the nature of any actual relationship between variables, be a reasonable approximation, or capture the relationship exactly.

Machine learning specifically relates to models that are *learned from data* as opposed to those built from knowledge about the variables

Generally, we'll focus on parameterized models, as parameterized models.

#### Overfitting & Underfitting

Overfitting: producing a model that performs very well on training data but does not generalize to other data (likely, what was 'learned' from training data was spurious)

Underfitting: producing a model that doesn't perform well on any data (e.g. tryingto fit a linear function to XOR).

The most fundemental approach to avoiding these issues together is partitioning of data into {train, valid, test} sets and comparing performance as needed.

An over-simplification:
- **train**: used to fit the models parameters
- **valid**: used to choose best hyper-parameters and model
- **test**: only evaluated for overall performance metric. important not to fit aggressively to this as it would then be roughly equivalent in purpose to the validation set

In [1]:
import random, copy
from typing import TypeVar, List, Tuple

# for validating our work
from sklearn import metrics

In [2]:
X = TypeVar('X')  # generic type to represent a data point

def split_data(data: List[X], proportion_train: float) -> Tuple[List[X], List[X]]:
    """
    Given a dataset `data` and a proportion `proportion_train` that should be the training set,
    return a tuple with the dataset split into train, test appropriately. Shallow-copies dataset.
    """
    data = copy.copy(data)  # shallow. data[:] also works for reference if you see it in the wild, but too implicit imo
    random.shuffle(data)
    cut = int(len(data) * proportion_train)
    return data[0:cut], data[cut:]

In [3]:
Y = TypeVar('Y')  # generic type to represent an output variable

def train_test_split(xs: List[X], ys: List[Y], proportion_train: float) -> Tuple[List[X], List[X], List[Y], List[Y]]:
    """
    Given a dataset `xs`  with corresponding outputs `ys` and a proportion `proportion_train` that should be 
    the training set,return a tuple with the dataset split into train, test appropriately. Shallow-copies dataset.
    """
    assert len(xs) == len(ys), 'nnumber of inputs and outputs should be the same'
    train_indices, test_indices = split_data(range(len(xs)), proportion_train)
    xs, ys = copy.copy(xs), copy.copy(ys)
    return ([xs[i] for i in train_indices],
            [xs[i] for i in test_indices],
            [ys[i] for i in train_indices],
            [ys[i] for i in test_indices])

##### Correctness

Accuracy can be a poor metric when there is significant class imbalance in a (e.g. binary) classification task. Consider this model for predicting {"Will develop Leukemia", "Will **never** develop Leukemia"}:

Model: `lambda user: 1 if user.name == 'Luke' else 0`

Given that *most people* will never develop Leukemia, and also are not named Luke, this model is ~98% 'accurate', despite clearly being wrong. We should break up predictions and correctness/errors like so instead:

1. True Positive: predicts "Will develop Leukemia" and user does in fact develop Leukemia
2. False Positive (Type 1 Error): predicts "Will develop Leukemia", but user *never develope Leukemia*
3. False Negative (Type 2 Error): predicts "Will **never** develop Leukemia", but user does in fact develop Leukemia
4. True Negative: predicts "Will **never** develop Leukemia", and user never develope Leukemia

##### Confusion Matrices

We can arrange this in a 'confusion matrix', starting with just counts. Lets assume:
- 5% of people are named Luke
- 1.4% of people develop Leukemia
- being named Luke and developing Leukemia are truly independent, despite our model's claim


|   |Leukemia|No Leukemia|Total|
|---|--------|-----------|-----|
|**"Luke"**|70|4,930|5,000|
|**"Not Luke"**|13,930|981,070|995,000|
|**Total**|14,000|986,000|1,000,000|

###### Some statistics about performance based on a confusion matrix
- **Accuracy**: correct predictions / total predictions
- **Precision**: the accuracy of only the positive predictions (how confident are we in a positive result?)
- **Recall**: the fraction of positives our model was able to find (how well do our positive results generalize/(in a somewhat backward way) how confident might we be in negative results?)

In [4]:
def accuracy(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int) -> float:
    """
    return the accuracy from confusion matrix data (correct / total)
    """
    correct: int = true_positives + true_negatives
    total: int = true_positives + true_negatives + false_positives + false_negatives
    return float(correct) / total

assert accuracy(70, 4930, 13930, 981070) == .98114

In [5]:
def precision(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int) -> float:
    """
    return the precision from confusion matrix data (true positives / total positive predictions). To be compatible with `accuracy`
    we'll preserve the signature despite not using the negative counts
    """
    total_positive_predictions: int = true_positives + false_positives
    return float(true_positives) / total_positive_predictions
# Note: can we mathematically compare this to the incident rate we observed for positives (getting Leukemia?) 
# in this case it is the same, suggesting we don't know anything more than given 'Luke'
assert precision(70, 4930, 13930, 981070) == .014

In [6]:
def recall(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int) -> float:
    """
    return the recall from confusion matrix data (true positives / total positives). To be compatible with `accuracy`
    we'll preserve the signature despite not using the negative counts
    """
    total_positive_outcomes: int = true_positives + false_negatives
    return float(true_positives) / total_positive_outcomes

assert recall(70, 4930, 13930, 981070) == .005

### F1 Score

We can combine `precision` and `recall` into a single metric called the **F1 Score**: the [harmonic mean](https://en.wikipedia.org/wiki/Harmonic_mean) between precision and recall. By definition, it must lie between them. 

Why harmonic mean? The F1 score is a composite measure that was hand-crafted to satisfy conditions specified [here](https://stackoverflow.com/questions/26355942/why-is-the-f-measure-a-harmonic-mean-and-not-an-arithmetic-mean-of-the-precision)

There exists an F0.5 and F2 score that reweights the importance toward better precision or recall in cases where one is more important than another using a parameter `beta` (e.g. in medicine, for a cheap test for something dangerous, recall might be more important than precision: we can illucidate false positives by re-testing, assuming factors leading to false positives are within the test and not the subject so that tests are independent).

In [7]:
def f1_score(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int) -> float:
    """
    compute the F1 score from the confusion matrix
    """
    p = precision(true_positives, false_positives, false_negatives, true_negatives)    
    r = recall(true_positives, false_positives, false_negatives, true_negatives)
    return 2 * p * r / (p + r)

##### F-beta score
We can also generalize by including the parameter beta, which we can default to 1

In [8]:
def f_beta_score(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int, 
                 beta: float = 1.) -> float:
    """
    compute the F-beta score from the confusion matrix
    """
    p = precision(true_positives, false_positives, false_negatives, true_negatives)    
    r = recall(true_positives, false_positives, false_negatives, true_negatives)
    return (1 + beta) * p * r  / ((beta * p) + r)

# lets refactor f1_score to use f_beta_score
def f1_score(true_positives: int, false_positives: int, false_negatives: int, true_negatives: int) -> float:
    """
    compute the F1 score from the confusion matrix
    """
    return f_beta_score(true_positives, false_positives, false_negatives, true_negatives, beta=1.)

### The Bias Variance Trade-off

**Bias**: a measure of how much our *choice of model* imposes mistakes on *any training set* drawn from the true population. (e.g. trying to fit a straight-line to stock prices has high-bias: we aren't likely to fit our own training data, or any other training data drawn from stock prices, with just a single straight line). Similarly, XOR is not linearly separable by ANY polynomial: we will never be able to fit it if our model is parameterized this way.

**Variance**: a measure of how much our *choice of model* imposes variance on predictions/parameters when we switch between *different training sets* drawn from the same true population. E.g. we can fit an N-degree polynomial to any collection of (N - 1) points, including stock prices. This would give a regression error in the training set of exactly zero, but a slight change in which points we choose to fit would significantly effect the chosen polynomial, and thus the variance in predicted outcomes

### Feature Selection & Extraction

**Feature**: an input point/dimension to our dataset (e.g. yesterdays closing price, patients Name).

As a rule-of-thumb, models fit to few features are likely to underfit (the un-represented features impact the data enough that we can't generalize from those we measured). Similarly, models fit to many features are likely to overfit (there is likely to be some spurious correlation in the training set between a subset of many features that doesn't actually generalize to the true population).

How many features is too-many, or too-few?

E.g. imagine we want to predict salary from *only* years of experience. We have only 1 feature, even if we tell our regression algorithm about several `(years_experience, years_experience ** 2 ... years_experience ** N)` for learning an N-degree polynomial.

Often, we need to extract/choose features (e.g. for spam classification, domain-of-sender, contains "Prince", contains "Viagra", contains "Winner")

Different selected features types (boolean, k-class, real number, etc.) are suitable for different modeling choices:

- Naive Bayes classifiers are suited to boolean features
- Regression models require numeric features (real numbers? e.g. booleans can be 1.0 or 0.0?)
- Decision trees handle numeric or categorical data (but make decisions based on comparison to a learned-parameter for numerics)